In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-contrib-python

In [24]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video, display
import os



def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny_edges = cv2.Canny(blur, 50, 150)
    return canny_edges

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(100, height), (1180, height), (600, 380)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 10)
    return line_image

def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1 * (3 / 5))
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        params = np.polyfit((x1, x2), (y1, y2), 1)
        slope, intercept = params
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    
    averaged_lines = []
    if left_fit:
        left_avg = np.mean(left_fit, axis=0)
        averaged_lines.append(make_coordinates(image, left_avg))
    if right_fit:
        right_avg = np.mean(right_fit, axis=0)
        averaged_lines.append(make_coordinates(image, right_avg))

    return np.array(averaged_lines)

def process_frame(frame):
    canny_image = canny(frame)
    cropped_image = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    if lines is not None:
        averaged_lines = average_slope_intercept(frame, lines)
        line_image = display_lines(frame, averaged_lines)
        combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        return combo_image
    return frame



os.makedirs("output", exist_ok=True)


input_path = "test_videos/solidYellowLeft.mp4"
output_path = "output/lane_detected_output.mp4"

cap = cv2.VideoCapture(input_path)


width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break

    lane_frame = process_frame(frame)
    out.write(lane_frame)

cap.release()
out.release()


